In [ ]:
from chi import server, context, lease
import os

context.version = "1.0" 
context.choose_project()
context.choose_site(default="CHI@UC") # change this according to lease server

In [ ]:
l = lease.get_lease(f"gpu_rtx6000_project_grp6") # change this lease name
l.show()

In [ ]:
username = os.getenv('USER') # all exp resources will have this prefix
s = server.Server(
    # f"node_{username}_project6", 
    f"node-project-group6", 
    reservation_id=l.node_reservations[0]["id"],
    image_name="CC-Ubuntu24.04-CUDA"
)
s.submit(idempotent=True)

In [4]:
s.associate_floating_ip()

ResourceError: None of the ports can route to floating ip 192.5.86.177 on server c2de2a4f-b80d-4f77-b76c-ab80cfa9b1d7

In [ ]:
s.refresh()
s.check_connectivity()

Checking connectivity to None port 22.


In [ ]:
s.execute("git clone https://github.com/teaching-on-testbeds/serve-model-chi")

In [ ]:
# setup docker
s.execute("curl -sSL https://get.docker.com/ | sudo sh")
s.execute("sudo groupadd -f docker; sudo usermod -aG docker $USER")

In [ ]:
# setup nvidia-container
s.execute("curl -fsSL https://nvidia.github.io/libnvidia-container/gpgkey | sudo gpg --dearmor -o /usr/share/keyrings/nvidia-container-toolkit-keyring.gpg \
  && curl -s -L https://nvidia.github.io/libnvidia-container/stable/deb/nvidia-container-toolkit.list | \
    sed 's#deb https://#deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://#g' | \
    sudo tee /etc/apt/sources.list.d/nvidia-container-toolkit.list")
s.execute("sudo apt update")
s.execute("sudo apt-get install -y nvidia-container-toolkit")
s.execute("sudo nvidia-ctk runtime configure --runtime=docker")
# for https://github.com/NVIDIA/nvidia-container-toolkit/issues/48
s.execute("sudo jq 'if has(\"exec-opts\") then . else . + {\"exec-opts\": [\"native.cgroupdriver=cgroupfs\"]} end' /etc/docker/daemon.json | sudo tee /etc/docker/daemon.json.tmp > /dev/null && sudo mv /etc/docker/daemon.json.tmp /etc/docker/daemon.json")
s.execute("sudo systemctl restart docker")

In [ ]:
# now connect with terminal: ssh -i ~/.ssh/pub_key_wl3187 cc@A.B.C.D